<a href="https://colab.research.google.com/github/nishkeni/Stanford_Artificial_Intelliegnce_NLP_NLU_ML/blob/master/BERT_lab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# BERT Lab

In 2018, models such as [BERT](https://arxiv.org/abs/1810.04805), [Open GPT](https://openai.com/blog/better-language-models/) and [ELMO](https://arxiv.org/abs/1802.05365) were released and performed well on many benchmark NLP tasks with minimal task-specific tuning. The publicly available pretrained models can be used either to extract high quality language features from your text data, or fine-tuned on a specific task (classification, entity recognition, etc.) with your data to produce [state of the art predictions](https://gluebenchmark.com/leaderboard).

In this notebook we will use a pre-trained BERT model to **extract features**, namely word embedding vectors, from text data. We will explore different ways of extracting these embeddings and build a simple binary classifier model to compare different embedding approaches.

Traditionally, words were converted to their vector representations by either **uniquely indexing (one-hot encoding)** them or using neural network based word embeddings such as **Word2Vec or Fasttext**. BERT offers an advantage over models like Word2Vec because while each word has a fixed representation in Word2Vec regardless of the context within which the word appears, BERT produces word representations that are dynamically informed by the words around them. For example, given two sentences:

<font color='green'>The man was accused of robbing a bank.</font>

<font color='green'>The man went fishing by the bank of the river.</font>

Models such as Word2Vec would produce the same word embeddings for the word _"bank"_ in both sentences (embeddings remain the same regardless of the context), while in BERT word embeddings for _"bank"_ would be different for  both sentences. 

*Note that this is in no way a detailed or a comprehensive explanation on BERT architecture (or its alternatives). It is a brief introduction to BERT and we explore how pre-trained BERT models can be used to extract word embeddings. We will use these embeddings for a simple classification task where we try to predict if a piece of text is a positive or a negative review.*

#### This notebook has two sections:
I. **BERT Embeddings Walk Through**:
    Here we explore how to extract embeddings from a pre-trained BERT model.

II. **Classification**:
    Here we build a base binary classification model (logisitic regression) for a binary classification task. Your task will be to try improving the model performance.


## I. BERT Embeddings Walk Through

### Format the input sentence

BERT expects input sentences to be in a specific format:

- Tokens to mark the beginning ([CLS]) and separation/end of sentences ([SEP])
- Word piece tokenization
- Segment IDs used to distinguish different sentences

![bert-input-format](https://i.imgur.com/Z8MXvQG.png)

In [0]:
# Install the required modules
!pip install transformers
!pip install pandas
!pip install numpy
!pip install tqdm
!pip install sklearn

In [0]:
# We are loading a smaller 'bert-base-uncased' model for this notebook. For more information on pre-trained models, check this: https://github.com/google-research/bert#pre-trained-models
import torch
from transformers import *

# model -> BertModel
# tokenizer -> BertTokenizer
# model name -> 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

#### Tokenization

In the example below, the word "ephemeral" is split into smaller subwords and characters. Hash signs preceding these subwords are the tokenizer's way of denoting that this subword or character is part of a larger word and preceded by another subword. 

The BERT tokenizer was created with a WordPiece model. This model greedily creates a fixed-size vocabulary of individual characters, subwords, and words that best fit our language data. To tokenize a word in this model, the tokenizer first checks if the whole word is in the vocabulary. If not, it tries to break the word into the largest possible subwords contained in the vocabulary, and as a last resort will decompose the word into individual characters. Note that because of this, we don't run into the problem of out of vocabulary words.

So, rather than assigning "ephemeral" and every other out of vocabulary word to something like 'UNK' (unknown vocabulary token), we split it into subword tokens ['ep', '##hem', '##eral'] that will retain some of the contextual meaning of the original word.


(For more information about WordPiece, see the [original paper](https://static.googleusercontent.com/media/research.google.com/en//pubs/archive/37842.pdf) and further disucssion in Google's [Neural Machine Translation System](https://arxiv.org/pdf/1609.08144.pdf).)



In [0]:
# Using the BERT tokenizer
example_text = "ephemeral"
print(tokenizer.tokenize(example_text))

In [0]:
# Adding tokens [CLS] and [SEP] at start and end of a sentence
text = "After stealing money from the bank vault, the bank robber was seen fishing on the Mississippi river bank."
marked_text = "[CLS] " + text + " [SEP]"

print (marked_text)

# Using BERT word piece tokenizer 
tokenized_text = tokenizer.tokenize(marked_text)
print ("\nAfter tokenization:\n",tokenized_text)

#### Segment IDs

BERT is trained on and expects sentence pairs (The [SEP] token is used to separate multiple sentences), using 1s and 0s to distinguish between the two sentences. 

That is, for each token in "tokenized_text," we must specify which sentence it belongs to: sentence 0 (a series of 0s) or sentence 1 (a series of 1s). For our purposes, single-sentence inputs only require a series of 1s, so we will create a vector of 1s for each token in our input sentence. 

If you want to process two sentences, assign each word in the first sentence plus the '[SEP]' token a 0, and all tokens of the second sentence a 1.

Essentially you can have multiple sentence pairs as shown below:

- **"<font color='green'>[CLS] The man was accused of robbing a bank. [SEP] The man was seen fishing by the river bank. [SEP]</font>"**
  - tokens in this case would be: ['[CLS]', 'the', 'man', 'was', 'accused', 'of', 'robb', '##ing', 'a', 'bank', '.', '[SEP]', 'the', 'man', 'was', 'seen', 'fishing', 'by', 'the', 'river', 'bank', '.', '[SEP]']
  - segment ids in this case would be: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
  

or you can have it as two separate sentences as shown below:

- **"<font color='green'>[CLS] The man was accused of robbing a bank. [SEP]</font>"**
  - tokens in this case would be: ['[CLS]', 'the', 'man', 'was', 'accused', 'of', 'robb', '##ing', 'a', 'bank', '.', '[SEP]']
  - segment ids in this case would be: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

- **"<font color='green'>[CLS] The man was seen fishing by the river bank. [SEP]</font>"**
  - tokens in this case would be: ['[CLS]', 'the', 'man', 'was', 'seen', 'fishing', 'by', 'the', 'river', 'bank', '.', '[SEP]']
  - segment ids in this case would be: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [0]:
# For our task we're using single sentence per input text
segments_ids = [0] * len(tokenized_text)
print (segments_ids)

#### Convert tokens to indices

We call the tokenizer to match the tokens against their indices in the tokenizer vocabulary

In [0]:
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)

for tup in zip(tokenized_text, indexed_tokens):
    print(tup)

### Forward pass the input sentence

1. The pre-trained BERT PyTorch interface expects the data be in torch tensor format so we first convert the segment_ids and tokenized text 
 
2. model.eval() puts our model in evaluation mode as opposed to training mode.

3. Calling `from_pretrained` will download the pre-trained model specified. When we load the `bert-base-uncased`, we see the definition of the model printed. This particular model has 13 layers (the forward pass of the model includes the outputs of the initial embedding in addition to the encoder states, so we get 13 layers).

In [0]:
import torch

# Load pre-trained model (weights)
model = BertModel.from_pretrained('bert-base-uncased',
                                  output_hidden_states=True)

# Put the model in "evaluation" mode, meaning feed-forward operation.
model.eval()

In [0]:
# Convert inputs to PyTorch tensors
tokens_tensor = torch.tensor([indexed_tokens])
segments_tensors = torch.tensor([segments_ids])

# Predict hidden states features for each layer
with torch.no_grad():
    encoded_layers = model(tokens_tensor, token_type_ids=segments_tensors)

In [0]:
# layers x batch_size x sequence length x hidden state dimension
n_layers, batch_size, sequence_length, hidden_state_dim = encoded_layers[-1], encoded_layers[0], encoded_layers[0][0], encoded_layers[0][0][0]
len(n_layers), len(batch_size), len(sequence_length), len(hidden_state_dim)

#### BERT output

The `encoded_layers` variable in the section above has four dimensions:

1. The layer number (13 layers)
2. The batch number (1 sentence)
3. The word / token number (number of tokens in our sentence)
4. Hidden state for that layer (768 features)

In [0]:
# Convert the hidden state embeddings into single token vectors

# Will have the shape: [# tokens, # layers, # features]
token_embeddings = [] 

batch_i = 0
# For each token in the sentence...
for token_i in range(len(tokenized_text)):
  
    # Holds 12 layers of hidden states for each token 
    hidden_layers = [] 

    # For each of the 12 layers...[last layer is excluded. See issue : https://github.com/huggingface/transformers/issues/1332]
    for layer_i in range(len(encoded_layers[-1][0:12])):

        # Lookup the vector for `token_i` in `layer_i`
        vec = encoded_layers[-1][layer_i][batch_i][token_i]

        hidden_layers.append(vec)
    
    token_embeddings.append(hidden_layers)

# Sanity check the dimensions:
print ("Number of tokens in sequence:", len(token_embeddings))
print ("Number of layers per token:", len(token_embeddings[0]))

### Create word and sentence embeddings


To extract features we'd like individual vectors for each of our tokens, or even a single vector representation of the whole sentence.

In order to get the individual vectors we can combine some of the layer vectors. BERT authors tested different combinations of this by feeding different vector combinations as input features to a BiLSTM used on a named entity recognition task and observing the resulting F1 scores.

![bert-feature-extraction-contextualized-embeddings](https://jalammar.github.io/images/bert-feature-extraction-contextualized-embeddings.png)

Using the last four layers produced the best results on this specific task. Many of the other methods returned close results, and so it is advisable to test different versions depending on the specific application.

#### Word vectors

Let's create word vectors by **using just the last layer**, to give us a single vector per token. Each of those vectors will have a length of `768`. 

In [0]:
# Every token vector : [sentence sequence length x 768]
token_vecs = [each_token[-1] for each_token in token_embeddings]

print('shape of token_vecs: %d x %d' % (len(token_vecs), len(token_vecs[0])))

Another way of creating word vectors is by **summing** together the last four layers.

In [0]:
# Stores the token vectors, with shape [sentence sequence length x 768]
token_vecs_sum = []

# For each token in the sentence...
for token in token_embeddings:
    # Sum the vectors from the last four layers.
    sum_vec = torch.sum(torch.stack(token)[-4:], 0)
    
    # Use `sum_vec` to represent `token`.
    token_vecs_sum.append(sum_vec)

print ('Shape is: %d x %d' % (len(token_vecs_sum), len(token_vecs_sum[0])))

#### Sentence vectors
To get a single vector for our entire sentence we have multiple application-dependent strategies, but a simple approach is to average the last hidden layer of each token, producing a single 768 length vector. 
<a href="https://stackoverflow.com/questions/29760935/how-to-get-vector-for-a-sentence-from-the-word2vec-of-tokens-in-sentence#answer-31738627">Here</a> are different ways of creating sentence vectors from word vectors (the link refers to Word2Vec embeddings, however the approach can be used here as well). 

In [0]:
sentence_embedding = torch.mean(encoded_layers[-1][11], 1)
print("shape of sentence embedding", list(sentence_embedding.shape))

### Confirming contextually dependent vectors

To confirm that the value of these vectors are in fact contextually dependent, let's take a look at the output from the following sentence:

In [0]:
print (text)

In [0]:
for i,x in enumerate(tokenized_text):
    print(i,x)

In [0]:
from sklearn.metrics.pairwise import cosine_similarity

# Compare "bank" as in "bank robber" to "bank" as in "river bank"
different_bank = cosine_similarity(token_vecs_sum[10].reshape(1,-1), token_vecs_sum[19].reshape(1,-1))[0][0]

# Compare "bank" as in "bank robber" to "bank" as in "bank vault" 
same_bank = cosine_similarity(token_vecs_sum[10].reshape(1,-1), token_vecs_sum[6].reshape(1,-1))[0][0]

In [0]:
print ("Similarity of 'bank' as in 'bank robber' to 'bank' as in 'bank vault':",  same_bank)

In [0]:
print ("Similarity of 'bank' as in 'bank robber' to 'bank' as in 'river bank':",  different_bank)

#### <font color='red'>Try for yourself</font>

Can you find one more example where the similarity of word embeddings for the same word in a similar context (such as 'bank' in 'bank robber' and 'bank' in 'bank vault') is greater than the similarity of word embeddings for same words in a different context (such as 'bank' in 'bank robber' and 'river bank').

## II. Classification

Next we'll use the `bert-base-uncased` pre-trained model and test the embeddings on a classification task. For this task we'll be using a food reviews dataset and build a small classification model to classify reviews as either positive or negative. 

For this exercise we take a look at two ways of creating word embeddings:
1. **Summing** together the hidden state of the last four layers. 
2. Using the hidden state of the last layer only.

Both of these methods would create a word embedding of shape 768. One simple way of creating **sentence embeddings** is to extract word vectors for each of the tokens in a given sentence and calculate the mean vector of all the token embeddings. This is how we'll be creating our sentence embeddings for this exercise.

### Data preparation

In [0]:
import pandas as pd
from tqdm import tqdm
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix
import numpy as np

In [0]:
def show_confusion_matrix(C, class_labels=['0', '1']):
    """
    :C: shape (2,2) as given by scikit-learn confusion_matrix function ['ndarray']
    :class_labels: list of strings, default simply labels 0 and 1 ['List']

    Draws confusion matrix with associated metrics.
    """
    import matplotlib.pyplot as plt

    assert C.shape == (2, 2), "Confusion matrix should be from binary classification only."

    # true negative, false positive, etc...
    tn = C[0, 0]
    fp = C[0, 1]
    fn = C[1, 0]
    tp = C[1, 1]

    NP = fn + tp  # Num positive examples
    NN = tn + fp  # Num negative examples
    N = NP + NN

    fig = plt.figure(figsize=(8, 8))
    ax = fig.add_subplot(111)
    ax.imshow(C, interpolation='nearest', cmap=plt.cm.gray)

    # Draw the grid boxes
    ax.set_xlim(-0.5, 2.5)
    ax.set_ylim(2.5, -0.5)
    ax.plot([-0.5, 2.5], [0.5, 0.5], '-k', lw=2)
    ax.plot([-0.5, 2.5], [1.5, 1.5], '-k', lw=2)
    ax.plot([0.5, 0.5], [-0.5, 2.5], '-k', lw=2)
    ax.plot([1.5, 1.5], [-0.5, 2.5], '-k', lw=2)

    # Set xlabels
    ax.set_xlabel('Predicted Label', fontsize=16)
    ax.set_xticks([0, 1, 2])
    ax.set_xticklabels(class_labels + [''])
    ax.xaxis.set_label_position('top')
    ax.xaxis.tick_top()
    # These coordinate might require some tinkering. Ditto for y, below.
    ax.xaxis.set_label_coords(0.34, 1.06)

    # Set ylabels
    ax.set_ylabel('True Label', fontsize=16, rotation=90)
    ax.set_yticklabels(class_labels + [''], rotation=90)
    ax.set_yticks([0, 1, 2])
    ax.yaxis.set_label_coords(-0.09, 0.65)

    # Fill in initial metrics: tp, tn, etc...
    ax.text(0, 0,
            'True Neg: %d\n(Num Neg: %d)' % (tn, NN),
            va='center',
            ha='center',
            bbox=dict(fc='w', boxstyle='round,pad=1'))

    ax.text(0, 1,
            'False Neg: %d' % fn,
            va='center',
            ha='center',
            bbox=dict(fc='w', boxstyle='round,pad=1'))

    ax.text(1, 0,
            'False Pos: %d' % fp,
            va='center',
            ha='center',
            bbox=dict(fc='w', boxstyle='round,pad=1'))

    ax.text(1, 1,
            'True Pos: %d\n(Num Pos: %d)' % (tp, NP),
            va='center',
            ha='center',
            bbox=dict(fc='w', boxstyle='round,pad=1'))

    # Fill in secondary metrics: accuracy, true pos rate, etc...
    ax.text(2, 0,
            'False Pos Rate: %.2f' % (fp / (fp + tn + 0.)),
            va='center',
            ha='center',
            bbox=dict(fc='w', boxstyle='round,pad=1'))

    ax.text(2, 1,
            'True Pos Rate: %.2f' % (tp / (tp + fn + 0.)),
            va='center',
            ha='center',
            bbox=dict(fc='w', boxstyle='round,pad=1'))

    ax.text(2, 2,
            'Accuracy: %.2f' % ((tp + tn + 0.) / N),
            va='center',
            ha='center',
            bbox=dict(fc='w', boxstyle='round,pad=1'))

    ax.text(0, 2,
            'Neg Pre Val: %.2f' % (1 - fn / (fn + tn + 0.)),
            va='center',
            ha='center',
            bbox=dict(fc='w', boxstyle='round,pad=1'))

    ax.text(1, 2,
            'Pos Pred Val: %.2f' % (tp / (tp + fp + 0.)),
            va='center',
            ha='center',
            bbox=dict(fc='w', boxstyle='round,pad=1'))

    plt.tight_layout()
    plt.show()

In [0]:
class Embeddings:
    LAST_LAYER = 1
    LAST_4_LAYERS = 2
    def __init__(self):
        self._tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        self._bert_model = BertModel.from_pretrained('bert-base-uncased', output_hidden_states=True)
        self._bert_model.eval()

    def tokenize(self, sentence):
        """

        :param sentence: input sentence ['str']
        :return: tokenized sentence based on word piece model ['List']
        """
        marked_sentence = "[CLS] " + sentence + " [SEP]"
        tokenized_text = self._tokenizer.tokenize(marked_sentence)
        return tokenized_text

    def get_bert_embeddings(self, sentence):
        """

        :param sentence: input sentence ['str']
        :return: BERT pre-trained hidden states (list of torch tensors) ['List']
        """
        # Predict hidden states features for each layer

        tokenized_text = self.tokenize(sentence)
        indexed_tokens = self._tokenizer.convert_tokens_to_ids(tokenized_text)

        segments_ids = [0] * len(tokenized_text)

        # Convert inputs to PyTorch tensors
        tokens_tensor = torch.tensor([indexed_tokens])
        segments_tensors = torch.tensor([segments_ids])

        with torch.no_grad():
            encoded_layers = self._bert_model(tokens_tensor, token_type_ids=segments_tensors)

        return encoded_layers[-1][0:12]

    def sentence2vec(self, sentence, layers):
        """

        :param sentence: input sentence ['str']
        :param layers: parameter to decide how word embeddings are obtained ['str]
            1. 'last' : last hidden state used to obtain word embeddings for sentence tokens
            2. 'last_4' : last 4 hidden states used to obtain word embeddings for sentence tokens

        :return: sentence vector [List]
        """
        encoded_layers = self.get_bert_embeddings(sentence)
        
        if layers == 1:
            # using the last layer embeddings
            token_embeddings = encoded_layers[-1]
            # summing the last layer vectors for each token
            sentence_embedding = torch.mean(token_embeddings, 1)
            return sentence_embedding.view(-1).tolist()

        elif layers == 2:
            token_embeddings = []
            tokenized_text = self.tokenize(sentence)

            batch_i = 0
            # For each token in the sentence...
            for token_i in range(len(tokenized_text)):

                # Holds 12 layers of hidden states for each token
                hidden_layers = []

                # For each of the 12 layers...
                for layer_i in range(len(encoded_layers)):
                    # Lookup the vector for `token_i` in `layer_i`
                    vec = encoded_layers[layer_i][batch_i][token_i]

                    hidden_layers.append(list(vec.numpy()))

                token_embeddings.append(hidden_layers)

            # using the last 4 layer embeddings
            token_vecs_sum = []

            # For each token in the sentence...
            for token in token_embeddings:
                # Sum the vectors from the last four layers.
                sum_vec = np.sum(token[-4:], axis=0)

                # Use `sum_vec` to represent `token`.
                token_vecs_sum.append(list(sum_vec))

            # summing the last layer vectors for each token
            sentence_embedding = np.mean(token_vecs_sum, axis=0)
            return sentence_embedding.ravel().tolist()


In [0]:
def model_grid_search(parameters, x_train, y_train, model, scoring):
    """

    :param parameters: hyperparameter choices to tune. Eg:
                    Hyperparameter values for logistic regression
                    {'loss' :['log'],
                    'penalty':['l1','l2','elasticnet'],
                    'alpha':[float(i)/10000000 for i in range(1,5,1)],
                    'n_jobs':[-1]}

                    [dict]

    :param x_train: training data (2D matrix) [List or ndarray]
    :param y_train: training data predictor values [List or ndarray]
    :param model: sklearn model to perform gridsearch on
    :param scoring: Finds the best model based on the scoring metric used (https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter)
    :return: grid search instance with information on all trained classifiers
    """
    clf = GridSearchCV(model, parameters, cv=5, scoring=scoring)
    clf.fit(x_train, y_train)
    return clf


def test_metrics(y_pred, y_test):
    """
    
    :param y_pred: predicted values [List]
    :param y_test: actual predictor values [List]
    
    :return precision, recall, F1 and confusion matrix scores [Dict]
    """
    conf_matrix = confusion_matrix(y_test, y_pred)

    # Precision and recall
    fp = conf_matrix[0, 1]
    fn = conf_matrix[1, 0]
    tp = conf_matrix[1, 1]

    precision = 100 * float(tp) / (tp + fp)
    recall = 100 * float(tp) / (tp + fn)
    F1 = 2 * (precision * recall) / (precision + recall)
    
    return {
        'precision': precision,
        'recall': recall,
        'F1': F1,
        'confusion_matrix': conf_matrix
    }

In [0]:
embed_model = Embeddings()

#### Dataset
This dataset consists of reviews of fine foods from Amazon. The original dataset spans a period of more than 10 years, including ~500,000 reviews up to October 2012. Reviews include product and user information, ratings, and a plain text review. It also includes reviews from all other Amazon categories. The `Score` column consists of scores ranging from 1 to 5 where **1** and **2** represent negative reviews, **4** and **5** represent positive reviews and a score of **3** mean the review is neutral. For simplicity we've combined scores **4,5** and tagged it as a positive score and combined scores **1,2** and tagged it as a negative score. Reviews with a score of **3** (neutal review) have not beeen incuded. The `Text` and `Summary` columns have been pre-processed. <a href="https://www.kaggle.com/snap/amazon-fine-food-reviews">Here</a> is a link to the original dataset.

We have provided a dataset with pre-preprocessed text and summary column. To obtain the sentence vector for our model, we'll be focusing on just the `Text` column. In the `Score` column a score of 1 indicates a positive review and a score of 0 indicates a negative review. The dataset is highly imbalanced with ~88 % of reviews belonging to the positive class (representend with a 1 in the `Score` column) and ~12 % belonging to the negative class (represented with a 0 in the `Score` column).

Run the below two cells to download the pre-processed train and test csv files. The dataset has been hosted on Google Drive and has been made available to anyone with the link. 

1. https://medium.com/@annissouames99/how-to-upload-files-automatically-to-drive-with-python-ee19bb13dda
2. Link to `train.csv` : https://drive.google.com/open?id=1IyCOmYJFILPkhx6ASvI0iJ_3Obvw1g0O
3. Link to `test.csv` : https://drive.google.com/open?id=1g0RoPXFUUgUGh4Mxa3QY_J_tz3LO_IqA

In [0]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
id_train = "1IyCOmYJFILPkhx6ASvI0iJ_3Obvw1g0O"
id_test = "1g0RoPXFUUgUGh4Mxa3QY_J_tz3LO_IqA"

df_train_downloaded = drive.CreateFile({'id':id_train}) 
df_train_downloaded.GetContentFile('train.csv')

df_test_downloaded = drive.CreateFile({'id':id_test}) 
df_test_downloaded.GetContentFile('test.csv')

In [0]:
# You should have train.csv and test.csv
ls

In [0]:
df_train = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")

In [0]:
df_train.head(3)

In [0]:
df_test.head(3)

### Modeling

We will be using logistic regression to build a baseline model. Logistic regression is a linear model for classification and a good option for building our baseline. We'll be using sklearn module to build our model. We also use the grid search method to find the best hyperparameters for the logistic regression model and observe the model performance. <a href="https://ayearofai.com/rohan-6-follow-up-statistical-interpretation-of-logistic-regression-e78de3b4d938">Here</a> is a small refresher on logistic regression. 

In [0]:
training_data_size = 5000  # you can experiment with more data to get a more realistic performance score. With a fewer datapoints the model tends to overfit

text_train = df_train['Text'].iloc[0:training_data_size].values
y_train = df_train['Score'].iloc[0:training_data_size].values

text_test = df_train['Text'].iloc[0:training_data_size].values
y_test = df_train['Score'].iloc[0:training_data_size].values

#### 1. BERT last layer hidden state as embeddings

In [0]:
x_train = []
for sentence in tqdm(text_train):
    x_train.append(embed_model.sentence2vec(sentence, layers=embed_model.LAST_LAYER))
    
x_test = []
for sentence in tqdm(text_test):
    x_test.append(embed_model.sentence2vec(sentence, layers=embed_model.LAST_LAYER))

In [0]:
parameters = {'loss' :['log'],'penalty':['l1','l2','elasticnet'],'alpha':[float(i)/10000000 for i in range(1,5,1)],'n_jobs':[-1]}
LR = SGDClassifier(fit_intercept=True, random_state=42)
scoring = 'recall'

In [0]:
clf = model_grid_search(parameters, x_train, y_train, model=LR, scoring=scoring)
print("Best params grid search", clf.best_params_)
y_pred = clf.predict(x_test)
metrics_dict = test_metrics(y_pred, y_test)
print("Precision", metrics_dict['precision'])
print("Recall", metrics_dict['recall'])
print("f1 score", metrics_dict['F1'])

In [0]:
show_confusion_matrix(metrics_dict['confusion_matrix'])

#### 2.  BERT last 4 layer hidden states as embeddings

In [0]:
x_train = []
for sentence in tqdm(text_train):
    x_train.append(embed_model.sentence2vec(sentence, layers=embed_model.LAST_4_LAYERS))
    
x_test = []
for sentence in tqdm(text_test):
    x_test.append(embed_model.sentence2vec(sentence, layers=embed_model.LAST_4_LAYERS))

In [0]:
parameters = {'loss' :['log'],'penalty':['l1','l2','elasticnet'],'alpha':[float(i)/10000000 for i in range(1,5,1)],'n_jobs':[-1]}
LR = SGDClassifier(fit_intercept=True, random_state=42)
scoring = 'recall'

In [0]:
LR = SGDClassifier(fit_intercept=True)
clf = model_grid_search(parameters, x_train, y_train, model=LR, scoring=scoring)
print("Best params grid search", clf.best_params_)
y_pred = clf.predict(x_test)
metrics_dict = test_metrics(y_pred, y_test)
print("Precision", metrics_dict['precision'])
print("Recall", metrics_dict['recall'])
print("f1 score", metrics_dict['F1'])

In [0]:
show_confusion_matrix(metrics_dict['confusion_matrix'])

## Things to try:

1. Try different non-linear classifiers (decision tree, random forest .etc) to achieve better results than the baseline.
2. Try different ways of building your word embeddings and use that on your desired classification model: (refer to the previous section [above](https://colab.research.google.com/drive/1g7OWTJKVWwij9Bit9nZDamaUI2--fjmO#scrollTo=ytonVXQWGcei))
    - Sum of all 12 layer hidden states (word embedding would be of size 768)
    - First layer hidden state (word embedding would be of size 768)
    - Second to last layer hidden state (word embedding would be of size 768)
    - Concat of last 4 layer hidden states (word embedding would be of size 768 * 4)
3. Use larger models such as `bert-large-uncased` instead of `bert-base-uncased`
3. You can also use `BERT as service`, to obtain <a href="https://github.com/hanxiao/bert-as-service#getting-elmo-like-contextual-word-embedding">sentence embeddings</a> (it supports many other features as well and their repo is pretty well documented) 

## References:
1. https://mccormickml.com/2019/05/14/BERT-word-embeddings-tutorial/
2. http://jalammar.github.io/illustrated-bert/
3. https://jalammar.github.io/illustrated-transformer/
4. https://towardsdatascience.com/nlp-extract-contextualized-embeddings-from-bert-keras-tf-67ef29f60a7b
5. [Annotated Transformer](https://nlp.seas.harvard.edu/2018/04/03/attention.html): a blogpost by Professor Sasha Rush describing the transformer architecture by implementing it from the paper in PyTorch.